# Benchamark CICADA performance versus supervised models

In [ ]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from scipy import interpolate
from scipy.linalg import norm
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
# Setup globals

MODELS_DIRECTORY = './models'
TRAINING = False
CV = 3
ZERO_BIAS_DATASET = '/eos/user/a/adpol/L1AD/Background/EphemeralZeroBias2018RunD_1.h5'

In [ ]:
# Setup seeds

np.random.seed(42)
tf.random.set_seed(42)

## Prepare the datasets

Load $0.5$~M ZeroBias samples as background.

In [ ]:
with h5py.File(ZERO_BIAS_DATASET, 'r') as h5:
    ZeroBias = h5['CaloRegions'][:500000].astype('float32')
print('ZeroBias shape: {}'.format(ZeroBias.shape))

Load up to $0.5$~M signal samples from each process.

In [ ]:
with h5py.File('/eos/user/a/adpol/L1AD/Signal/120X/GluGluHToTauTau_M-125_TuneCP5_14TeV.h5', 'r') as h5:
    GluGluHToTauTau = h5['CaloRegions'][:500000].astype('float32')
print('GluGluHToTauTau shape: {}'.format(GluGluHToTauTau.shape))

with h5py.File('/eos/user/a/adpol/L1AD/Signal/120X/GluGluToHHTo4B_node_cHHH1_TuneCP5_14TeV.h5', 'r') as h5:
    GluGluToHHTo4B = h5['CaloRegions'][:500000].astype('float32')
print('GluGluToHHTo4B shape: {}'.format(GluGluToHHTo4B.shape))

with h5py.File('/eos/user/a/adpol/L1AD/Signal/120X/TT_TuneCP5_14TeV.h5', 'r') as h5:
    TTBar = h5['CaloRegions'][:500000].astype('float32')
print('TTBar shape: {}'.format(TTBar.shape))

with h5py.File('/eos/user/a/adpol/L1AD/Signal/120X/haa4b_ma15_powheg.h5', 'r') as h5:
    Haa4b = h5['CaloRegions'][:500000].astype('float32')
print('Haa4b shape: {}'.format(Haa4b.shape))

with h5py.File('/eos/user/a/adpol/L1AD/Signal/120X/SUEP_L1_NOPU.h5', 'r') as h5:
    SUEP = h5['CaloRegions'][:500000].astype('float32')
    # Overlay SUEP samples with ZeroBias for pileup
    randomevent = np.random.randint(low=0, high=500000, size=500000)
    SUEP += ZeroBias[randomevent][:SUEP.shape[0]]
print('SUEP shape: {}'.format(SUEP.shape))

Generate synthetic samples.

In [ ]:
def gaussian_kernel(size, length=3, sigma=0.5, min_pu=20, max_pu=100):
    pu = np.random.uniform(min_pu, max_pu, size=size)
    eta = np.random.randint(0, 14-length+1, size=size)
    phi = np.random.randint(0, 18-length+1, size=size)
    ax = np.linspace(-(length - 1) / 2., (length - 1) / 2., length)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    regions = np.zeros((size, 18, 14))
    for s,(e,p,k) in enumerate(zip(eta, phi, np.repeat(pu, 9).reshape(-1,3,3) * kernel / np.max(kernel))):
        regions[s,p:p+3, e:e+3] = k
    return regions

synthetic = gaussian_kernel(100000)

In [ ]:
def plot_deposits(m):
    fig, ax = plt.subplots(figsize=(16, 12))
    im = ax.imshow(m)
    ax.set_ylabel(r'$\phi$', color='black', size=20)
    ax.set_xlabel(r'$\eta$', color='black', size=20)
    ax.set_xticks([])
    ax.set_yticks([])
    clb = fig.colorbar(im)
    clb.ax.tick_params(labelsize=20) 
    clb.ax.set_title(r'$E_T$',fontsize=20)
    plt.title("Region deposits", size=20)
    plt.show(fig)
    
plot_deposits(synthetic[0])
plot_deposits(synthetic[1])

Split the datasets for fixed size training size.

In [ ]:
train_size=40000 # upper-bound of SUEP samples

ZeroBias_train, ZeroBias_test = train_test_split(
    ZeroBias, train_size=train_size, random_state=42, shuffle=True)

GluGluHToTauTau_train, GluGluHToTauTau_test = train_test_split(
    GluGluHToTauTau, train_size=train_size, random_state=42, shuffle=True)

GluGluToHHTo4B_train, GluGluToHHTo4B_test = train_test_split(
    GluGluToHHTo4B, train_size=train_size, random_state=42, shuffle=True)

TTBar_train, TTBar_test = train_test_split(
    TTBar, train_size=train_size, random_state=42, shuffle=True)

Haa4b_train, Haa4b_test = train_test_split(
    Haa4b, train_size=train_size, random_state=42, shuffle=True)

SUEP_train, SUEP_test = train_test_split(
    SUEP, train_size=train_size, random_state=42, shuffle=True)

synthetic_train, synthetic_test  = train_test_split(
    synthetic, train_size=train_size, random_state=42, shuffle=True)

Generate a blend of signals for training

In [ ]:
blend_train = np.vstack([
    GluGluHToTauTau_train[:8000],
    GluGluToHHTo4B_train[:8000],
    TTBar_train[:8000],
    Haa4b_train[:8000],
    SUEP_train[:8000]
])

## Train the supervised models

Define the model that has similar size to the CICADA.v2 student. The only difference is 2 unit output with softmax activation. The softmax does not have to be implemented in hardware so the total FPGA resource usage and latency will be very simialar.

In [ ]:
def get_model():
    return keras.Sequential(
        [
            layers.Conv2D(filters=3, kernel_size=(3, 3), strides=2, padding='valid', activation='relu', input_shape=(18,14,1)),
            layers.Flatten(),
            layers.Dense(units=20, activation='relu'),
            layers.Dense(units=2, activation = 'softmax')
        ]
    )

The training uses three callbacks: early stopping, reduce on plateu and model checkpointing. Batch size, validation split and learning rate is fixed across models.

In [ ]:
def train_nn(model, X, y, batch_size, loss, name, validation_split=0.0, metrics=None):
    
    early_stopper = EarlyStopping(monitor="val_loss",
                                  patience=64,
                                  verbose=True,
                                  mode="auto")
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.7,
                                  patience=5)

    checkpoint_callback = ModelCheckpoint(("%s/%s.h5" % (MODELS_DIRECTORY, name)),
                                          monitor="val_loss",
                                          verbose=False,
                                          save_weights_only=False,
                                          save_best_only=True,
                                          mode="min")

    opt = keras.optimizers.Adam(learning_rate=0.001,
                                beta_1=0.9,
                                beta_2=0.999,
                                epsilon=None,
                                decay=0.0,
                                amsgrad=False)

    model.compile(loss=loss, metrics=metrics, optimizer=opt)

    return model.fit(X, y,
                     batch_size=batch_size,
                     epochs=8192,
                     verbose=0,
                     initial_epoch=0,
                     shuffle=True,
                     validation_split=validation_split,
                     callbacks=[early_stopper, reduce_lr, checkpoint_callback])

Define helper function to plot loss and accuracy

In [ ]:
def plot_loss(training):
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch", horizontalalignment='right', x=1.0)
    ax1.set_ylabel("Loss", horizontalalignment='right', y=1.0)
    ax1.set_yscale('log')
    ax1.tick_params(axis='y', labelcolor='red')
    ax1.plot(training.history['loss'],
             label='Training')
    ax1.plot(training.history['val_loss'],
             label='Validation')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Accuracy', color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    ax2.plot(training.history['val_accuracy'],
             color='green',
             label='Val Accuracy')
    ax1.legend(bbox_to_anchor=(1.1, 1.00), loc='upper left')
    ax2.legend(bbox_to_anchor=(1.1, 0.00), loc='lower left')
    plt.show(fig)

Train the supervised model and save them to directory. The blend model has an issue with generalization. It will likely have to have either more capacity, experience or both to understand difference between the background and low-PT (and high-PT) signal samples.

In [ ]:
def supervised_training(signal, name):

    model = get_model()

    X_train = np.expand_dims(np.append(ZeroBias_train, signal, axis=0), axis=-1)
    y_train = np.append(np.zeros(len(ZeroBias_train)), np.ones(len(signal)), axis=0)
    y_train = keras.utils.to_categorical(y_train, 2)

    training = train_nn(
        model,
        X_train,
        y_train,
        2048,
        "categorical_crossentropy",
        name,
        validation_split=0.2,
        metrics="accuracy")

    plot_loss(training)

if TRAINING:
    for i in range(CV):
        supervised_training(GluGluHToTauTau_train, "m_GluGluHToTauTau-{}".format(i))
        supervised_training(GluGluToHHTo4B_train, "m_GluGluToHHTo4B-{}".format(i))
        supervised_training(TTBar_train, "m_TTBar-{}".format(i))
        supervised_training(Haa4b_train, "m_Haa4b-{}".format(i))
        supervised_training(SUEP_train, "m_SUEP-{}".format(i))
        supervised_training(blend_train, "m_blend-{}".format(i))
        supervised_training(synthetic_train, "m_synthetic-{}".format(i))

Load CICADA models.

In [ ]:
cicada_v1 = models.load_model('./ML_CMSL1CaloTrigger/saved_models/model_sA')
cicada_v2 = models.load_model('./ML_CMSL1CaloTrigger/saved_models/qmodel_oct24')

## Evaluation

Four metrics to collect: ROC AUC, efficiency at 5kHz, efficiency at 3 kHz, efficiency at 1 kHz. Each model is evaluated on all test datasets.

In [ ]:
models = [r"$\mathcal{M}_{H \rightarrow \tau \tau}$",
          r"$\mathcal{M}_{\rm{SM }~HH \rightarrow 4b}$",
          r"$\mathcal{M}_{T{\bar{T}}}$",
          r"$\mathcal{M}_{H \rightarrow aa \rightarrow 4b}$",
          r"$\mathcal{M}_{\rm{SUEP}}$",
          r"$\mathcal{M}_{\rm{BLEND}}$",
          r"$\mathcal{M}_{\rm{Synthetic}}$",
          r"$\mathcal{M}_{\rm{Baseline}}$",
          r"$\mathcal{M}_{\rm{CICADA.v1}}$",
          r"$\mathcal{M}_{\rm{CICADA.v2}}$"]
datasets = [r"$\mathcal{D}_{H \rightarrow \tau \tau}$",
            r"$\mathcal{D}_{\rm{SM }~HH \rightarrow 4b}$",
            r"$\mathcal{D}_{T{\bar{T}}}$",
            r"$\mathcal{D}_{H \rightarrow aa \rightarrow 4b}$",
            r"$\mathcal{D}_{\rm{SUEP}}$",
            r"$\mathcal{D}_{\rm{Synthetic}}$"]

In [ ]:
matrix_auc = np.zeros((6, 10))
matrix_eff_5hz = np.zeros((6, 10))
matrix_eff_3hz = np.zeros((6, 10))
matrix_eff_1hz = np.zeros((6, 10))

matrix_auc_std = np.zeros((6, 10))
matrix_eff_5hz_std = np.zeros((6, 10))
matrix_eff_3hz_std = np.zeros((6, 10))
matrix_eff_1hz_std = np.zeros((6, 10))

for x_idx, signal in enumerate(
    [GluGluHToTauTau_test,
     GluGluToHHTo4B_test,
     TTBar_test,
     Haa4b_test,
     SUEP_test,
     synthetic_test]):

    X_test = np.expand_dims(np.append(ZeroBias_test, signal, axis=0), axis=-1)
    y_test = np.append(np.zeros(len(ZeroBias_test)), np.ones(len(signal)), axis=0)

    for y_idx, name in enumerate(
        ["m_GluGluHToTauTau",
         "m_GluGluToHHTo4B",
         "m_TTBar",
         "m_Haa4b",
         "m_SUEP",
         "m_blend",
         "m_synthetic"]):

        roc_auc, eff5hz, eff3hz, eff1hz = [], [], [], []
        for i in range(CV):
            model = get_model()
            model.load_weights("{}/{}-{}.h5".format(MODELS_DIRECTORY, name, i))

            fpr, tpr, _ = roc_curve(y_test, model.predict(X_test)[:,-1])

            roc_auc.append(auc(fpr, tpr))
            interpolated = interpolate.interp1d(fpr*28.61, tpr)
            eff5hz.append(interpolated(0.005))
            eff3hz.append(interpolated(0.003))
            eff1hz.append(interpolated(0.001))
        
        matrix_auc[x_idx][y_idx] = np.mean(roc_auc)
        matrix_auc_std[x_idx][y_idx] = np.std(roc_auc)
        matrix_eff_5hz[x_idx][y_idx] = np.mean(eff5hz)
        matrix_eff_5hz_std[x_idx][y_idx] = np.std(eff5hz)
        matrix_eff_3hz[x_idx][y_idx] = np.mean(eff3hz)
        matrix_eff_3hz_std[x_idx][y_idx] = np.std(eff3hz)
        matrix_eff_1hz[x_idx][y_idx] = np.mean(eff1hz)
        matrix_eff_1hz_std[x_idx][y_idx] = np.std(eff1hz)

    # Add baseline
    pileup = np.mean(ZeroBias_train, axis=0)
    fnorm = norm(X_test-pileup.reshape(18, 14, 1), ord=None, axis=(1,2), keepdims=False)
    fpr, tpr, _ = roc_curve(y_test, fnorm)
    roc_auc = auc(fpr, tpr)
    interpolated = interpolate.interp1d(fpr*28.61, tpr)
    eff5hz = interpolated(0.005)
    eff3hz = interpolated(0.003)
    eff1hz = interpolated(0.001)
    matrix_auc[x_idx][7] = roc_auc
    matrix_eff_5hz[x_idx][7] = eff5hz
    matrix_eff_3hz[x_idx][7] = eff3hz
    matrix_eff_1hz[x_idx][7] = eff1hz

    # Add CICADA_V1
    score = cicada_v1.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, score)
    roc_auc = auc(fpr, tpr)
    interpolated = interpolate.interp1d(fpr*28.61, tpr)
    eff5hz = interpolated(0.005)
    eff3hz = interpolated(0.003)
    eff1hz = interpolated(0.001)
    matrix_auc[x_idx][8] = roc_auc
    matrix_eff_5hz[x_idx][8] = eff5hz
    matrix_eff_3hz[x_idx][8] = eff3hz
    matrix_eff_1hz[x_idx][8] = eff1hz

    # Add CICADA_V2
    score = cicada_v2.predict(X_test.reshape(-1, 252))
    fpr, tpr, _ = roc_curve(y_test, score)
    roc_auc = auc(fpr, tpr)
    interpolated = interpolate.interp1d(fpr*28.61, tpr)
    eff5hz = interpolated(0.005)
    eff3hz = interpolated(0.003)
    eff1hz = interpolated(0.001)
    matrix_auc[x_idx][9] = roc_auc
    matrix_eff_5hz[x_idx][9] = eff5hz
    matrix_eff_3hz[x_idx][9] = eff3hz
    matrix_eff_1hz[x_idx][9] = eff1hz

In [ ]:
def plot_matrix(matrix, matrix_std, title):
    fig, ax = plt.subplots(figsize=(16, 12))
    im = ax.imshow(matrix, alpha=0.7, cmap='RdYlGn', vmin=np.min(matrix), vmax=np.max(matrix))
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(models)), labels=models)
    ax.set_yticks(np.arange(len(datasets)), labels=datasets)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    for i in range(len(datasets)):
        for j in range(len(models)):
            if j < 7:
                text = ax.text(j, i,
                               """{0:.4f}

$\pm$ {1:.4f}""".format(matrix[i, j], matrix_std[i, j]),
                               ha="center", va="center", color="black", size=14)
            else:
                text = ax.text(j, i, "{0:.4f}".format(matrix[i, j]),
                               ha="center", va="center", color="black", size=14)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    fig.tight_layout()
    plt.title(title, fontsize=20)
    plt.show()

In [ ]:
plot_matrix(matrix_auc, matrix_auc_std, "ROC AUC")

In [ ]:
plot_matrix(matrix_eff_5hz, matrix_eff_5hz_std, "Efficiency 5kHz")

In [ ]:
plot_matrix(matrix_eff_3hz, matrix_eff_3hz_std, "Efficiency 3kHz")

In [ ]:
plot_matrix(matrix_eff_1hz, matrix_eff_1hz_std, "Efficiency 1kHz")